# How Agents use Tools

In this notebook we will take a beginners introduction into how AI Agents actually use tools.

I'm hoping this will demystify the misconception that AI use tools themselves and are able to execute code and terminal commands. This is not the case as we will see soon.

### Importing Libraries

In [1]:
import os
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set")

client = OpenAI()

### Basic chat with an LLM

In [2]:
messages = [
    {"role":"assistant", "content":"You are a helpful assistant"},
    {"role":"user", "content":"What is the weather in Abu Dhabi?"}
]

In [3]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = messages
)

message = response.choices[0].message
print(message)

ChatCompletionMessage(content='I don\'t have real-time weather data. However, you can easily check the current weather in Abu Dhabi by using a weather website, a weather app, or searching "Abu Dhabi weather" on a search engine. Typically, Abu Dhabi has a hot desert climate, so it\'s often warm or hot throughout much of the year.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


The response we get as a ChatCompletionMessage object is as follows:

ChatCompletionMessage(content="I can't provide real-time weather updates. However, you can easily check the weather in Abu Dhabi by using a weather website or app for the most accurate and current information. Is there anything else you would like to know?", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

Notice the `content` and `tool_calls` attributes.

Looking at the content, the model isn't able to get the current weather info.

Let's give it a tool so it can look up the weather.

The below python function calls the Open-meteo weather API to grab the details of the weather at a particular location given a latitude and longitude.

### Defining a function/tool

In [4]:
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

Now that we have our function, let's let the model know that it has a tool that it can use to answer the users query.

We first need to define a tool schema which is a specially structured json which tells the model exactly what the function is, the description, what are the inputs or arguments of the function and what is it's purpose.

A simple structure is as follows.

```json
{
  "type": "function",

    "function": {

        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {

            "type": "object",
            "properties": {

                "latitude": {"type": "number"},
                "longitude": {"type": "number"}

            },
            "required": ["latitude","longitude"],
            "additionalProperties": False

        },

        "strict": True
    }
}
```

The above json schema defines what function is and what are the properties (arguments) of the function.

This is used by the model as context to know what tools are available to it and details of each tool.

This is passed into the `tools` parameter of the chat completions api endpoint as a list of all tool schemas available to the model.

Lets pass this information to our model.

### Defining the function schema

In [5]:
tools = [{
  "type": "function",

    "function": {

        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {

            "type": "object",
            "properties": {

                "latitude": {"type": "number"},
                "longitude": {"type": "number"}

            },
            "required": ["latitude","longitude"],
            "additionalProperties": False

        },
        
        "strict": True
    }
}]

Lets just print the entire message object from the model and see all the items we receive.

### Giving our LLM tools

In [6]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = messages,
    tools = tools
)

message = response.choices[0].message
print(message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_cDl43I9m4s6e1K2GViF89oSW', function=Function(arguments='{"latitude":24.4534,"longitude":54.3773}', name='get_weather'), type='function')])


Result:

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_KKdmlg85rzBV1qm47dI9AdsM', function=Function(arguments='{"latitude":24.4534,"longitude":54.1861}', name='get_weather'), type='function')])

The chat completions object no longer has any content for us, but there is something in the tools call attribute.

The `tool_call` attribute contains a ChatCompletionsMessageToolCall object which has a `function` attribute giving us the details of the tool the LLM wants us to use.

The model **can not** execute the tool itself, it provides us the name and arguments of the tools that need to be executed and **we** have to go and actually execute that tool.

So the next step is for us to grab the details of the tool, execute the function ourselves, and provide the model with the response of that tool.

In [7]:
message.tool_calls

[ChatCompletionMessageToolCall(id='call_cDl43I9m4s6e1K2GViF89oSW', function=Function(arguments='{"latitude":24.4534,"longitude":54.3773}', name='get_weather'), type='function')]

In [8]:
tool_call = message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(result)

24.9


The temperature in Abu Dhabi is 25.2 according to the result of the tool call.

This now needs to be formatted into a tool call result message and passed into the messages list as follows.

In [9]:
messages.append(message)            # append the models tool call message
messages.append({                   # append result message
    "role":"tool",
    "tool_call_id":tool_call.id,
    "content":str(result)
})

pprint(messages)

[{'content': 'You are a helpful assistant', 'role': 'assistant'},
 {'content': 'What is the weather in Abu Dhabi?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_cDl43I9m4s6e1K2GViF89oSW', function=Function(arguments='{"latitude":24.4534,"longitude":54.3773}', name='get_weather'), type='function')]),
 {'content': '24.9',
  'role': 'tool',
  'tool_call_id': 'call_cDl43I9m4s6e1K2GViF89oSW'}]


In [10]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = messages,
    tools = tools
)

message = response.choices[0].message.content
print(message)

The current temperature in Abu Dhabi is 24.9°C.
